<a href="https://colab.research.google.com/github/Syed-Raza-Ali/Human-into-loop/blob/main/human_into_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**HUMAN INTO LOOP**
- **Under Construction**

In [ ]:
# Install all necessary packages
!pip install -q langchain-core langchain-google-genai langgraph python-dotenv tavily-python langchain_community

In [ ]:
# API KEYS VARIABLES
import os
from google.colab import userdata

os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")
GOOGLE_API_KEY = userdata.get("GEMINI_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults


# Initialize Tavily search tool
tavily_search = TavilySearchResults(max_results = 3)

# Maintain chat history
chat_history = []

def stream_graph_updates(user_input):
    """A placeholder function for processing graph updates."""
    print(f"Processing graph updates for: {user_input}")


def main():
  """ Get realtime information using tavily search """

  while True:


    try:
        # Prompt the user for input
        print("\n")
        user_input = input("User : ").strip()

        # Exit condition
        if user_input.lower() in ["quit", "exit", "q","bye"]:
            print("Bye! Have a great day!")
            break

        # Handle empty input
        if not user_input:
            print("No prompt found. Please type something.")
            continue

        # Add user input to chat history
        chat_history.append({"role": "user", "content": user_input})

        # Call the placeholder function (replace as needed)
        try:
            stream_graph_updates(user_input)
        except NameError:
            # Handle missing function gracefully
            print("stream_graph_updates is not implemented. Skipping.")

        # Perform a Tavily search if the function is not implemented
        search_docs = tavily_search.invoke(user_input)

        # Add Tavily response to chat history
        chat_history.append({"role": "assistant", "content": str(search_docs)})

        # Display results from Tavily search
        print("Search Results:")
        for i, doc in enumerate(search_docs, start=1):
            print(f"\nResult {i}:")
            print(f"Here is the {i} documentaion from web search you can get your answer from here:")
            print(f"Documentation: {doc.get('content', 'N/A')}")
            print(f"URL: {doc.get('url', 'N/A')}")

    except Exception as e:
        print(f"An error occurred: {e}")
        break

    print("\nChat History:")
    for message in chat_history:
      role = message["role"]
      content = message["content"]
      print(f"{role.capitalize()}: {content}")



main()

In [ ]:
import os
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.graph.state import CompiledStateGraph
from langgraph.checkpoint.memory import MemorySaver
from google.colab import userdata


GOOGLE_API_KEY = userdata.get("GEMINI_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY


# Disable LangSmith Tracing programmatically
langchain_api_key = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = "Health-Agent"

# Initialize the Gemini LLM model with Google API Key
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key = GOOGLE_API_KEY
)


# LangGraph nodes
def model_conversation(state: MessagesState):
    """Core conversational logic."""
    system_message = ""
    messages = [SystemMessage(content=system_message)] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": response}


# Define the workflow
workflow = StateGraph(MessagesState)
workflow.add_node("conversation", model_conversation)
workflow.add_edge(START, "conversation")
workflow.add_edge("conversation", END)


# Compile the graph with memory
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)


# Chatbot runtime logic
def agent_calling():
    """Run the agent."""
    config = {"configurable": {"thread_id": "1"}}

    # print("Hello dear, i am your MindCare Assistant, My expertise is about mental health")
    # print("Note: I have been designed exclusively as a MindCare assistant.i will not able to response any other questions")

    while True:
        user_input = input("You: ")

        if user_input.lower() in ["exit", "bye", "quit","q"]:
            print("Goodbye! Thank you for your trust, I hope this information is helpful for you")
            break


        input_message = HumanMessage(content=user_input)

        try:
            output = graph.invoke({"messages": [input_message]}, config)
        except Exception as e:
            print(f"Error: {str(e)}")
            continue

        bot_response = output["messages"][-1].content
        print(f"Chatbot: {bot_response}")

        # take snapshot for memory
        state = graph.get_state(config)
        # print(state)



# Run the chatbot
agent_calling()

